In [26]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from utils import read_config, MySQLAgent
import time, random

In [134]:
config = read_config('.env/connections.json')
sql_agent = MySQLAgent(config['VM1_mysql_conn_info'])

In [3]:
config['VM1_mysql_conn_info']

{'user': 'aiadmin',
 'pw': 'aichat2030!',
 'host': '138.3.214.21',
 'port': 3306,
 'database': 'CrawlerDB'}

In [4]:
query = """
select distinct company_name
from company

"""

df_company = sql_agent.read_table(query=query)
df_company

,company_name
0,國昌綜合工廠有限公司
1,華聯食品工業股份有限公司
2,建寶食品股份有限公司
3,財團法人台北市瑠公農業產銷基金會
4,新光醫療財團法人
...,...
11099,尚鱻農漁特產行
11100,容昇發商行
11101,豪富企業社
11102,華秝茶油純釀油坊


In [198]:
# df_company.to_csv('files/company_name.csv', index=False, encoding='utf-16')

In [5]:
query = """
select 公司代號 as stock_num, 營利事業統一編號 as busi_acc, 公司名稱 as company_name
from listed_otc_company
"""

otc = sql_agent.read_table(query=query)
otc.head()

,stock_num,busi_acc,company_name
0,1101,11913502,臺灣水泥股份有限公司
1,1102,3244509,亞洲水泥股份有限公司
2,1103,11892801,嘉新水泥股份有限公司
3,1104,7568009,環球水泥股份有限公司
4,1108,40601248,幸福水泥股份有限公司


In [6]:
otc[otc['stock_num'] == '1101'].busi_acc[0]

'11913502'

In [7]:
query = """
select *
from mops_season_report_copy
"""

fr = sql_agent.read_table(query=query)
fr

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq,period_year,season
0,流動資產,0,0,0,0,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,1,113,2
1,現金及約當現金,1136975,3.98,840325,2.99,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,2,113,2
2,透過損益按公允價值衡量之金融資產－流動,14619,0.05,57577,0.2,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,3,113,2
3,透過其他綜合損益按公允價值衡量之金融資產－流動,2869282,10.04,2392367,8.5,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,4,113,2
4,按攤銷後成本衡量之金融資產－流動,131334,0.46,102367,0.36,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,5,113,2
...,...,...,...,...,...,...,...,...,...,...,...,...
32947,本期現金及約當現金增加（減少）數,283887,0,164587,0,CashFlowStatement,9944,新麗企業股份有限公司,2024-07-16,60,113,2
32948,期初現金及約當現金餘額,2106903,0,1999100,0,CashFlowStatement,9944,新麗企業股份有限公司,2024-07-16,61,113,2
32949,期末現金及約當現金餘額,2390790,0,2163687,0,CashFlowStatement,9944,新麗企業股份有限公司,2024-07-16,62,113,2
32950,資產負債表帳列之現金及約當現金,1910157,0,2163687,0,CashFlowStatement,9944,新麗企業股份有限公司,2024-07-16,63,113,2


In [8]:
fr.report_name.unique()

array(['BalanceSheet', 'ProfitAndLose', 'CashFlowStatement'], dtype=object)

In [27]:
company_list = pd.read_csv('company_data.csv')
company_list.head()

,business_accounting_no,company_name,internal_id,query_name
0,700019,國昌綜合工廠有限公司,K7913,國昌綜合工廠
1,713181,華聯食品工業股份有限公司,J5984,華聯食品
2,784930,建寶食品股份有限公司,K4395,建寶食品
3,833117,高立展業有限公司,Q0717,高立展業
4,966299,財團法人台北市瑠公農業產銷基金會,K9232,台北市瑠公農業產銷基金會


In [9]:
# 網站連線資料
BalanceSheetURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb03";      # 資產負債表
ProfitAndLoseURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb04";    # 損益表
CashFlowStatementURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb05"; # 現金流量表

In [10]:
query = """
select distinct company_name
from company
where business_accounting_no = '35412204'
"""

company_list = sql_agent.read_table(query=query)
company_list

,company_name
0,華新麗華股份有限公司


In [96]:
stock_list = otc[otc['company_name'].isin(set(company_list.company_name))].stock_num.unique()
print("num of stock in comapny_list:", len(stock_list))
df_frc = pd.DataFrame()
error_stock_list = []

num of stock in comapny_list: 1


In [97]:
# Balancesheet crawler
for stock_number in stock_list:
       
    # stock_number = '1231'
    year = 113
    season = 1
    url = BalanceSheetURL

    form_data = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'co_id':stock_number,
        'year': year,
        'season': season,
    }
    
    try:
        r = requests.post(url,form_data)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find_all('table')[1]
        headers = [header.text for header in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            cells_text = [cell.text.strip() for cell in cells]
            rows.append(cells_text)

        # columns = ['acct_name', 'this_year_amt', 'this_year_percent', 'last_hf_amt', 'last_hf_amt_%', 'last_year_amt', 'last_year_percent']
        columns = ['acct_name', 'this_year_amt', 'this_year_percent', 'last_hf_amt', 'last_hf_amt_%', 'last_year_amt', 'last_year_percent', 'last_year_hf_amt', 'last_year_hf_percent']
        # Create a DataFrame
        df_temp = pd.DataFrame(rows, columns=columns)
        df_select = df_temp.iloc[:, [0,1,2,5,6]].iloc[3:].copy()
        
        df_select['report_name'] = 'BalanceSheet'
        df_select['company_id'] = otc[otc['stock_num'] == str(stock_number)].stock_num.values[0]
        df_select['company_name'] = otc[otc['stock_num'] == str(stock_number)].company_name.values[0]
        df_select['creation_date'] = datetime.now().strftime('%Y-%m-%d')
        df_select['seq'] = range(1, len(df_select) + 1)
            

        df_select['stock'] = stock_number

        df_frc = pd.concat([df_frc, df_select], axis=0)
        
        print(f'get {stock_number}')
        
        # time.sleep(random.uniform(10, 35))
         
    except requests.RequestException as e:
        error_stock_list.append(stock_number)
        print(f'Failed to retrieve data for stock number {stock_number}: {e}')
    except ValueError as e:
        error_stock_list.append(stock_number)
        print(f'Error parsing data for stock number {stock_number}: {e}')
    except Exception as e:
        error_stock_list.append(stock_number)
        print(f'An unexpected error occurred for stock number {stock_number}: {e}')
        

get 1605


In [98]:
df_frc_2 = df_frc.drop(['stock'], axis=1)

In [99]:
df_frc_2

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq
3,流動資產,,,,,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,1
4,現金及約當現金,"12,313,790",4.63,"8,257,051",3.23,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,2
5,透過損益按公允價值衡量之金融資產－流動,"1,339,835",0.50,"150,209",0.06,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,3
6,按攤銷後成本衡量之金融資產－流動,737,0.00,"1,509",0.00,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,4
7,避險之金融資產－流動,"9,427",0.00,"15,460",0.01,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,5
...,...,...,...,...,...,...,...,...,...,...
70,非控制權益,"9,927,706",3.74,"10,708,783",4.20,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,68
71,權益總額,"151,479,608",57.02,"135,742,714",53.18,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,69
72,負債及權益總計,"265,675,434",100.00,"255,273,437",100.00,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,70
73,預收股款（權益項下）之約當發行股數（單位：股）,0,,0,,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,71


In [16]:
conf = config['VM1_mysql_conn_info']
conf

{'user': 'aiadmin',
 'pw': 'aichat2030!',
 'host': '138.3.214.21',
 'port': 3306,
 'database': 'CrawlerDB'}

In [17]:
host = conf['host']
database = conf['database']
user = conf['user']
password = conf['pw']
port = conf['port']

In [19]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [35]:
try:
    connection = mysql.connector.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    if connection.is_connected():
        cursor = connection.cursor()
        
        for index, row in df_frc_2.iterrows():
            sql = """INSERT INTO mops_season_report_copy (acct_name, this_year_amt, this_year_percent, last_year_amt, last_year_percent, report_name, company_id, company_name, creation_date, seq)
                     VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
            cursor.execute(sql, tuple(row))
            connection.commit()
        
        print("Data inserted successfully")
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Data inserted successfully
MySQL connection is closed


In [100]:
df_frc_2

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq
3,流動資產,,,,,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,1
4,現金及約當現金,"12,313,790",4.63,"8,257,051",3.23,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,2
5,透過損益按公允價值衡量之金融資產－流動,"1,339,835",0.50,"150,209",0.06,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,3
6,按攤銷後成本衡量之金融資產－流動,737,0.00,"1,509",0.00,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,4
7,避險之金融資產－流動,"9,427",0.00,"15,460",0.01,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,5
...,...,...,...,...,...,...,...,...,...,...
70,非控制權益,"9,927,706",3.74,"10,708,783",4.20,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,68
71,權益總額,"151,479,608",57.02,"135,742,714",53.18,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,69
72,負債及權益總計,"265,675,434",100.00,"255,273,437",100.00,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,70
73,預收股款（權益項下）之約當發行股數（單位：股）,0,,0,,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,71


In [103]:
sql_agent.write_table(data=df_frc_2, table_name='mops_season_report_copy', index=False, if_exists='append', data_type=None)

In [21]:
# try:
#     connection = mysql.connector.connect(
#         host=host,
#         database=database,
#         user=user,
#         password=password
#     )
#     if connection.is_connected():
#         cursor = connection.cursor()
#         table_name = 'mops_season_report_copy'

#         # Retrieve column names
#         cursor.execute(f"DESCRIBE {table_name}")
#         columns = cursor.fetchall()

#         # Generate and execute ALTER TABLE statements
#         for column in columns:
#             column_name = column[0]
#             alter_statement = f"ALTER TABLE {table_name} MODIFY {column_name} VARCHAR(255)"
#             cursor.execute(alter_statement)
#             connection.commit()
        
#         print("All columns converted to VARCHAR(255) successfully")
# except Error as e:
#     print("Error while connecting to MySQL", e)
# finally:
#     if connection.is_connected():
#         cursor.close()
#         connection.close()
#         print("MySQL connection is closed")

All columns converted to VARCHAR(255) successfully
MySQL connection is closed


In [83]:
# df_frc.to_csv('fr_result.csv', index=False)
df_frc_balance = pd.read_csv('balance_result.csv')

In [84]:
df_frc_balance.nunique()

acct_name             156
this_year_amt        9439
this_year_percent    3217
last_year_amt        9368
last_year_percent    3278
report_name             1
company_id            209
company_name          209
creation_date           1
seq                    94
stock                 209
dtype: int64

In [85]:
df_frc_balance = pd.concat([df_frc_balance, df_frc_2], axis=0)

In [86]:
df_frc_balance = df_frc_balance.drop(['stock'], axis=1)

In [87]:
df_frc_balance.company_id.nunique()

210

In [122]:
df_profitloss = pd.DataFrame()

In [123]:
# Profit and loss
for stock_number in stock_list:
       
    # stock_number = '1231'
    year = 113
    season = 1
    url = ProfitAndLoseURL

    form_data = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'co_id':stock_number,
        'year': year,
        'season': season,
    }
    
    try:
        r = requests.post(url,form_data)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find_all('table')[1]
        headers = [header.text for header in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            cells_text = [cell.text.strip() for cell in cells]
            rows.append(cells_text)

        columns = ['acct_name', 'this_year_amt', 'this_year_percent', 'last_year_amt', 'last_year_percent']
        # Create a DataFrame
        df_temp = pd.DataFrame(rows, columns=columns)
        df_select = df_temp.iloc[3:].copy()
        
        df_select['report_name'] = 'ProfitAndLose'
        df_select['company_id'] = otc[otc['stock_num'] == str(stock_number)].stock_num.values[0]
        df_select['company_name'] = otc[otc['stock_num'] == str(stock_number)].company_name.values[0]
        df_select['creation_date'] = datetime.now().strftime('%Y-%m-%d')
        df_select['seq'] = range(1, len(df_select) + 1)
            
        df_profitloss = pd.concat([df_profitloss, df_select], axis=0)
        
        print(f'get {stock_number}')
        
        # time.sleep(random.uniform(10, 35))
         
    except requests.RequestException as e:
        error_stock_list.append(stock_number)
        print(f'Failed to retrieve data for stock number {stock_number}: {e}')
    except ValueError as e:
        error_stock_list.append(stock_number)
        print(f'Error parsing data for stock number {stock_number}: {e}')
    except Exception as e:
        error_stock_list.append(stock_number)
        print(f'An unexpected error occurred for stock number {stock_number}: {e}')

get 1605


In [124]:
df_profitloss.head()

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq
3,營業收入合計,"40,549,837",100.00,"50,713,578",100.00,ProfitAndLose,1605,華新麗華股份有限公司,2024-07-18,1
4,營業成本合計,"37,891,440",93.44,"46,192,595",91.09,ProfitAndLose,1605,華新麗華股份有限公司,2024-07-18,2
5,營業毛利（毛損）,"2,658,397",6.56,"4,520,983",8.91,ProfitAndLose,1605,華新麗華股份有限公司,2024-07-18,3
6,營業毛利（毛損）淨額,"2,658,397",6.56,"4,520,983",8.91,ProfitAndLose,1605,華新麗華股份有限公司,2024-07-18,4
7,營業費用,,,,,ProfitAndLose,1605,華新麗華股份有限公司,2024-07-18,5


In [67]:
# df_profitloss.to_csv('profitloss_result.csv', index=False, encoding='utf-16')
df_profitloss_all = pd.read_csv('profitloss_result.csv', encoding='utf-16')

In [68]:
df_profitloss_all.report_name.unique()

array(['ProfitAndLose'], dtype=object)

In [75]:
df_profitloss_all_1605 = pd.concat([df_profitloss_all, df_profitloss], axis=0)

In [79]:
df_profitloss_all_1605.company_id.nunique()

205

In [128]:
df_cash = pd.DataFrame()

In [130]:
# Cashflow
for stock_number in stock_list:
       
    # stock_number = '1231'
    year = 113
    season = 1
    url = CashFlowStatementURL

    form_data = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'co_id':stock_number,
        'year': year,
        'season': season,
    }
    
    try:
        r = requests.post(url,form_data)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find_all('table')[1]
        headers = [header.text for header in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            cells_text = [cell.text.strip() for cell in cells]
            rows.append(cells_text)

        columns = ['acct_name', 'this_year_amt', 'last_year_amt']
        # Create a DataFrame
        df_temp = pd.DataFrame(rows, columns=columns)
        df_select = df_temp.iloc[3:].copy()
        
        df_select['report_name'] = 'CashFlowStatement'
        df_select['company_id'] = otc[otc['stock_num'] == str(stock_number)].stock_num.values[0]
        df_select['company_name'] = otc[otc['stock_num'] == str(stock_number)].company_name.values[0]
        df_select['creation_date'] = datetime.now().strftime('%Y-%m-%d')
        df_select['seq'] = range(1, len(df_select) + 1)
            
        df_cash = pd.concat([df_cash, df_select], axis=0)
        
        print(f'get {stock_number}')
        
        # time.sleep(random.uniform(10, 35))
         
    except requests.RequestException as e:
        error_stock_list.append(stock_number)
        print(f'Failed to retrieve data for stock number {stock_number}: {e}')
    except ValueError as e:
        error_stock_list.append(stock_number)
        print(f'Error parsing data for stock number {stock_number}: {e}')
    except Exception as e:
        error_stock_list.append(stock_number)
        print(f'An unexpected error occurred for stock number {stock_number}: {e}')

get 1605


In [118]:
df_cash.head()

,acct_name,this_year_amt,last_year_amt,report_name,company_id,company_name,creation_date,seq
3,營業活動之現金流量－間接法,,,CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,1
4,繼續營業單位稅前淨利（淨損）,"671,948","3,001,904",CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,2
5,本期稅前淨利（淨損）,"671,948","3,001,904",CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,3
6,折舊費用,"1,896,229","1,590,586",CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,4
7,攤銷費用,"328,756","257,232",CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,5


In [61]:
# df_cash.to_csv('cash_result.csv', index=False, encoding='utf-16')
# df_cash_all = pd.read_csv('cash_result.csv', encoding='utf-16')

In [62]:
df_cash_all.report_name.unique()

array(['CashFlowStatement'], dtype=object)

In [80]:
df_cash_all.company_id.nunique()

208

In [81]:
df_cash_all_1605 = pd.concat([df_cash_all, df_cash], axis=0)

In [106]:
df_cash_all.head()

,acct_name,this_year_amt,last_year_amt,report_name,company_id,company_name,creation_date,seq,stock
0,營業活動之現金流量－間接法,NaN,NaN,CashFlowStatement,7568009,環球水泥股份有限公司,2024-07-16,1,1104
1,繼續營業單位稅前淨利（淨損）,"351,086","315,487",CashFlowStatement,7568009,環球水泥股份有限公司,2024-07-16,2,1104
2,本期稅前淨利（淨損）,"351,086","315,487",CashFlowStatement,7568009,環球水泥股份有限公司,2024-07-16,3,1104
3,折舊費用,"47,185","44,550",CashFlowStatement,7568009,環球水泥股份有限公司,2024-07-16,4,1104
4,攤銷費用,"1,006",890,CashFlowStatement,7568009,環球水泥股份有限公司,2024-07-16,5,1104


In [ ]:
df_cash_all_1605
df_profitloss_all_1605
df_frc_balance

In [139]:
df_season_all_1605 = pd.concat([df_frc_2, df_profitloss, df_cash], axis=0)

In [140]:
df_season_all_1605['period_year'] = '113'
df_season_all_1605['season'] = '1'

In [141]:
df_season_all_1605 = df_season_all_1605.fillna(0)
df_season_all_1605

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq,period_year,season
3,流動資產,,,,,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,1,113,1
4,現金及約當現金,"12,313,790",4.63,"8,257,051",3.23,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,2,113,1
5,透過損益按公允價值衡量之金融資產－流動,"1,339,835",0.50,"150,209",0.06,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,3,113,1
6,按攤銷後成本衡量之金融資產－流動,737,0.00,"1,509",0.00,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,4,113,1
7,避險之金融資產－流動,"9,427",0.00,"15,460",0.01,BalanceSheet,1605,華新麗華股份有限公司,2024-07-18,5,113,1
...,...,...,...,...,...,...,...,...,...,...,...,...
70,匯率變動對現金及約當現金之影響,"1,380,631",0,"39,246",0,CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,68,113,1
71,本期現金及約當現金增加（減少）數,"-4,033,222",0,"-11,181,708",0,CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,69,113,1
72,期初現金及約當現金餘額,"16,347,012",0,"19,438,759",0,CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,70,113,1
73,期末現金及約當現金餘額,"12,313,790",0,"8,257,051",0,CashFlowStatement,1605,華新麗華股份有限公司,2024-07-18,71,113,1


In [142]:
sql_agent.write_table(data=df_season_all_1605, table_name='mops_season_report_copy', index=False, if_exists='append', data_type=None)

In [25]:
df_test = fr.head()

In [37]:
# read balacne
df_balance = pd.read_csv('balance_result.csv')
# read profitloss
df_profitloss = pd.read_csv('profitloss_result.csv', encoding='utf-16')
# read cash
df_cash = pd.read_csv('cash_result.csv', encoding='utf-16')

In [33]:
df_balance.head()

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq,stock
0,流動資產,NaN,NaN,NaN,NaN,BalanceSheet,7568009,環球水泥股份有限公司,2024-07-13,1,1104
1,現金及約當現金,"1,136,975",3.98,"840,325",2.99,BalanceSheet,7568009,環球水泥股份有限公司,2024-07-13,2,1104
2,透過損益按公允價值衡量之金融資產－流動,"14,619",0.05,"57,577",0.20,BalanceSheet,7568009,環球水泥股份有限公司,2024-07-13,3,1104
3,透過其他綜合損益按公允價值衡量之金融資產－流動,"2,869,282",10.04,"2,392,367",8.50,BalanceSheet,7568009,環球水泥股份有限公司,2024-07-13,4,1104
4,按攤銷後成本衡量之金融資產－流動,"131,334",0.46,"102,367",0.36,BalanceSheet,7568009,環球水泥股份有限公司,2024-07-13,5,1104


In [34]:
df_profitloss.head()

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq,stock
0,營業收入合計,"1,908,572",100.00,"1,835,530",100.00,ProfitAndLose,7568009,環球水泥股份有限公司,2024-07-16,1,1104
1,營業成本合計,"1,525,149",79.91,"1,490,975",81.23,ProfitAndLose,7568009,環球水泥股份有限公司,2024-07-16,2,1104
2,營業毛利（毛損）,"383,423",20.09,"344,555",18.77,ProfitAndLose,7568009,環球水泥股份有限公司,2024-07-16,3,1104
3,營業毛利（毛損）淨額,"383,423",20.09,"344,555",18.77,ProfitAndLose,7568009,環球水泥股份有限公司,2024-07-16,4,1104
4,營業費用,NaN,NaN,NaN,NaN,ProfitAndLose,7568009,環球水泥股份有限公司,2024-07-16,5,1104


In [148]:
df_all = pd.concat([df_balance, df_profitloss, df_cash], axis=0).fillna(0)

In [149]:
df_all['period_year'] = '113'
df_all['season'] = '2'
df_all['company_id'] = df_all['stock']
df_all = df_all.drop(['stock'], axis=1)

In [150]:
df_all.head()

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq,period_year,season
0,流動資產,0,0,0,0,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,1,113,2
1,現金及約當現金,"1,136,975",3.98,"840,325",2.99,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,2,113,2
2,透過損益按公允價值衡量之金融資產－流動,"14,619",0.05,"57,577",0.2,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,3,113,2
3,透過其他綜合損益按公允價值衡量之金融資產－流動,"2,869,282",10.04,"2,392,367",8.5,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,4,113,2
4,按攤銷後成本衡量之金融資產－流動,"131,334",0.46,"102,367",0.36,BalanceSheet,1104,環球水泥股份有限公司,2024-07-13,5,113,2


In [152]:
# Remove commas and convert to numeric, forcing errors to NaN
df_all['this_year_amt'] = pd.to_numeric(df_all['this_year_amt'].str.replace(',', ''), errors='coerce').fillna(0).astype(int)
df_all['last_year_amt'] = pd.to_numeric(df_all['last_year_amt'].str.replace(',', ''), errors='coerce').fillna(0).astype(int)

In [56]:
df_test

,report_name,company_id,company_name,period_year,season,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,creation_date,seq
0,BalanceSheet,1101,台泥,112,1,流動資產,0.0,0.00,0.0,0.00,2024-04-10 13:45:47,1
1,BalanceSheet,1101,台泥,112,1,現金及約當現金,70957209.0,15.37,88842494.0,19.28,2024-04-10 13:45:47,2
2,BalanceSheet,1101,台泥,112,1,透過損益按公允價值衡量之金融資產－流動,648145.0,0.14,611802.0,0.13,2024-04-10 13:45:47,3
3,BalanceSheet,1101,台泥,112,1,透過其他綜合損益按公允價值衡量之金融資產－流動,6377062.0,1.38,5934753.0,1.29,2024-04-10 13:45:47,4
4,BalanceSheet,1101,台泥,112,1,按攤銷後成本衡量之金融資產－流動,30795636.0,6.67,20954299.0,4.55,2024-04-10 13:45:47,5


In [73]:
pd.concat([df_test, df_all.head()], axis=0)

,report_name,company_id,company_name,period_year,season,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,creation_date,seq
0,BalanceSheet,1101,台泥,112,1,流動資產,0.0,0.0,0.0,0.0,2024-04-10 13:45:47,1
1,BalanceSheet,1101,台泥,112,1,現金及約當現金,70957209.0,15.37,88842494.0,19.28,2024-04-10 13:45:47,2
2,BalanceSheet,1101,台泥,112,1,透過損益按公允價值衡量之金融資產－流動,648145.0,0.14,611802.0,0.13,2024-04-10 13:45:47,3
3,BalanceSheet,1101,台泥,112,1,透過其他綜合損益按公允價值衡量之金融資產－流動,6377062.0,1.38,5934753.0,1.29,2024-04-10 13:45:47,4
4,BalanceSheet,1101,台泥,112,1,按攤銷後成本衡量之金融資產－流動,30795636.0,6.67,20954299.0,4.55,2024-04-10 13:45:47,5
0,BalanceSheet,1104,環球水泥股份有限公司,113,2,流動資產,0.0,0.0,0.0,0.0,2024-07-13,1
1,BalanceSheet,1104,環球水泥股份有限公司,113,2,現金及約當現金,"1,136,975",3.98,"840,325",2.99,2024-07-13,2
2,BalanceSheet,1104,環球水泥股份有限公司,113,2,透過損益按公允價值衡量之金融資產－流動,"14,619",0.05,"57,577",0.2,2024-07-13,3
3,BalanceSheet,1104,環球水泥股份有限公司,113,2,透過其他綜合損益按公允價值衡量之金融資產－流動,"2,869,282",10.04,"2,392,367",8.5,2024-07-13,4
4,BalanceSheet,1104,環球水泥股份有限公司,113,2,按攤銷後成本衡量之金融資產－流動,"131,334",0.46,"102,367",0.36,2024-07-13,5


In [166]:
from sqlalchemy.types import VARCHAR, INTEGER, DECIMAL, DATETIME

data_type = {
    "report_name": VARCHAR(50),
    "company_id": VARCHAR(4),
    "company_name": VARCHAR(50),
    "period_year": INTEGER(),
    "season": INTEGER(),
    "acct_name": VARCHAR(100),
    "this_year_amt": INTEGER(),
    "this_year_percent": DECIMAL(20, 6),
    "last_year_amt": INTEGER(),
    "last_year_percent": DECIMAL(20, 6),
    "creation_date": DATETIME(),
    "seq": INTEGER()
}

In [188]:
query = """
select *
from mops_season_report_copy
"""

fr_copy = sql_agent.read_table(query=query)
fr_copy

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,company_id,company_name,creation_date,seq,period_year,season


In [153]:
df_final = pd.concat([fr_copy, df_all], axis=0)

In [154]:
df_final.dtypes

report_name          object
company_id           object
company_name         object
period_year          object
season               object
acct_name            object
this_year_amt         int64
this_year_percent    object
last_year_amt         int64
last_year_percent    object
creation_date        object
seq                   int64
dtype: object

In [155]:
def format_with_commas(x):
    return f"{x:,}"

df_final['this_year_percent'] = df_final['this_year_percent'].apply(format_with_commas)

ValueError: Cannot specify ',' with 's'.

In [175]:
df_final['this_year_amt'] = df_final['this_year_amt'].astype(str)
df_final['last_year_amt'] = df_final['last_year_amt'].astype(str)

In [ ]:
df_select_1 = df_final[df_final['period_year'] == '113' &]

In [190]:
sql_agent.write_table(df_all, table_name='mops_season_report', if_exists='append', index=None, data_type=None)

DataError: (pymysql.err.DataError) (1366, "Incorrect decimal value: '1,048.80' for column 'last_year_percent' at row 4012")
[SQL: INSERT INTO mops_season_report (acct_name, this_year_amt, this_year_percent, last_year_amt, last_year_percent, report_name, company_id, company_name, creation_date, seq, period_year, season) VALUES (%(acct_name)s, %(this_year_amt)s, %(this_year_percent)s, %(last_year_amt)s, %(last_year_percent)s, %(report_name)s, %(company_id)s, %(company_name)s, %(creation_date)s, %(seq)s, %(period_year)s, %(season)s)]
[parameters: [{'acct_name': '流動資產', 'this_year_amt': 0, 'this_year_percent': 0, 'last_year_amt': 0, 'last_year_percent': 0, 'report_name': 'BalanceSheet', 'company_id': 1104, 'company_name': '環球水泥股份有限公司', 'creation_date': '2024-07-13', 'seq': 1, 'period_year': '113', 'season': '2'}, {'acct_name': '現金及約當現金', 'this_year_amt': 1136975, 'this_year_percent': 3.98, 'last_year_amt': 840325, 'last_year_percent': 2.99, 'report_name': 'BalanceSheet', 'company_id': 1104

In [184]:
df_all.iloc[6190]

acct_name                    流動資產合計
this_year_amt            2452767394
this_year_percent             42.38
last_year_amt            1995727521
last_year_percent             39.55
report_name            BalanceSheet
company_id                     2330
company_name         台灣積體電路製造股份有限公司
creation_date            2024-07-13
seq                              12
period_year                     113
season                            2
Name: 6190, dtype: object